In [7]:
# 首先在服务器上clone当前仓库
!rm -rf application_benchmark
!ssh riscv-server git clone https://github.com/LiuZhetan/application_benchmark.git

Cloning into 'application_benchmark'...


In [15]:
# 然后进入仓库目录并执行测试脚本
# 该脚本会在服务器上执行redis的测试，并将测试结果保存在redis/results/目录下
!ssh riscv-server 'cd application_benchmark && sudo -S bash ./redis/run_benchmark.sh -p riscv'
# 从远程服务器中下载redis/results/目录下的测试结果

set cpus
mount: /cpusets: none already mounted or mount point busy.
       dmesg(1) may have more information after failed mount system call.
cset: **> mount of cpuset filesystem failed, do you have permission?
Redis is running
Measuring latency:
mount: /cpusets: none already mounted or mount point busy.
       dmesg(1) may have more information after failed mount system call.
Measuring intrinsic latency......
mount: /cpusets: none already mounted or mount point busy.
       dmesg(1) may have more information after failed mount system call.
......Running benchmark......
......Finished Benchmark.....
mount: /cpusets: none already mounted or mount point busy.
       dmesg(1) may have more information after failed mount system call.
cset: **> mount of cpuset filesystem failed, do you have permission?


In [16]:
# 在本地测试脚本
!echo xxxxx | sudo -S bash run_benchmark.sh

[sudo] password for ubuntu_wsl: Redis is not running, starting Redis
Measuring latency:
Measuring intrinsic latency, it takes 80 seconds......
Max latency so far: 2 microseconds.
Max latency so far: 42 microseconds.
Max latency so far: 44 microseconds.
Max latency so far: 47 microseconds.
Max latency so far: 62 microseconds.
Max latency so far: 256 microseconds.
Max latency so far: 882 microseconds.
Max latency so far: 1054 microseconds.
Max latency so far: 2075 microseconds.
Max latency so far: 2466 microseconds.

2690098655 total runs (avg latency: 0.0297 microseconds / 29.74 nanoseconds per run).
Worst run took 82922x longer than the average latency.
......Running benchmark......
1
"PING_INLINE","2500000.00","0.247","0.072","0.247","0.383","0.447","0.487"
"PING_MBULK","3333333.25","0.171","0.072","0.175","0.247","0.279","0.295"
"SET","2500000.00","0.269","0.080","0.279","0.439","0.503","0.551"
"GET","1428571.38","0.494","0.112","0.279","2.623","2.751","2.791"
"INCR","1666666.62","0.